In [8]:
import sys
# sys.path.append('/home/aih/shrey.parikh/PDAC/PDAC/code')
import scanpy as sc
import os
import warnings ; warnings.warn = lambda *args,**kwargs: None
import pandas as pd
import numpy as np
os.chdir('/home/aih/shrey.parikh/PDAC/PDAC')
import shutil
import pickle
from distutils.dir_util import copy_tree

# Prepare scpoli

In [10]:
def process_datasets(directories, concatenated_dataset_path, extension):
    adata_dataset = sc.read_h5ad(concatenated_dataset_path)
    for directory in directories:
        for file in os.listdir(directory):
            adata_training = adata_dataset #.copy()
            if file.endswith(extension):
                file_path = os.path.join(directory, file)
                target_dir = os.path.join('datasets_metrics', directory)
                if not os.path.exists(target_dir):
                    os.makedirs(target_dir)
                save_path = os.path.join(target_dir, file.replace('.h5ad', '_metrics.h5ad'))
                print(f'Processing {file_path.split('/')[1]}')
                adata = sc.read_h5ad(file_path)
                barcode = adata_training.obs_names.tolist()
                adata_training.obs_names = adata_training.obs.pseudo_barcode
                adata.obs_names = adata.obs.pseudo_barcode
                adata.obs_names = adata.obs_names.astype(str)
                adata.obs_names_make_unique()
                adata.obs['scpoli_labels'] = adata.obs.Label_Harmonized.fillna('Unknown')
                adata.obs = adata.obs.astype(str)
                adata.obs.loc[adata.obs['scpoli_labels'] == 'Unknown', 'scpoli_labels'] = adata.obs.loc[adata.obs['scpoli_labels'] == 'Unknown', 'cell_type_pred']
                adata = adata[adata_training.obs_names, :]
                adata = adata[~adata.obs.Dataset.str.contains('prototype')]
                adata_training.obsm['X_emb'] = adata.X
                adata_training.obsm['X_umap_emb'] = adata.obsm['X_umap']
                adata_training.obsm['X_umap_raw'] = adata_training.obsm['X_umap']
                adata_training.obs = adata.obs.copy()
                adata_training.uns = adata.uns.copy()
                adata_training.obsp = adata.obsp.copy()
                adata_training.uns['output_type'] = 'embed'
                adata_training.obs['Condition'] = np.where(adata_training.obs.Dataset == 'Regev', 'snRNA-seq', 'scRNA-seq')
                adata_training.obs['cell_barcode'] = barcode
                adata_training.obs.rename(columns={'pseudo_barcode': 'pseudo_barcode_two'}, inplace=True)
                # adata_training.obs = adata_training.obs.astype(str)
                keys_to_keep = ['X_emb', 'X_umap_emb', 'X_umap_raw']
                keys_to_delete = [key for key in adata.obsm.keys() if key not in keys_to_keep]
                for key in keys_to_delete:
                    del adata.obsm[key]
                print(adata_training)
                print(f'Saving at {save_path}')
                adata_training.write(save_path)

In [11]:
#modified scPoli because integration was wrong 6/4/24
#modified scPoli beacuse ordering of cells between training and output adata was different
process_datasets(['scPoli'], concatenated_dataset_path='concatenated_datasets/PDAC_concat_hvg_batch_key_samples_hvg.h5ad',
extension='donor_reference_latent_label_transfer.h5ad')

Processing scPoli_donor_reference_latent_label_transfer.h5ad
AnnData object with n_obs × n_vars = 837983 × 3781
    obs: 'Dataset', 'ID', 'pseudo_barcode_two', 'Level 1 Annotation', 'Level 2 Annotation', 'Level 3 Annotation', 'Cell_type', 'scanvi_labels', 'Label_Harmonized', 'conditions_combined', 'cell_type_pred', 'cell_type_uncert', 'classifier_outcome', 'batch', 'Label_Harmonized_pred', 'Label_Harmonized_uncert', 'query', 'scpoli_labels', 'Condition', 'cell_barcode'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'neighbors', 'umap', 'output_type'
    obsm: 'X_cnv', 'X_cnv_pca', 'X_cnv_umap', 'X_harmony', 'X_pca', 'X_umap', 'X_emb', 'X_umap_emb', 'X_umap_raw'
    layers: 'counts', 'raw'
    obsp: 'connectivities', 'distances'
Saving at datasets_metrics/scPoli/scPoli_donor_reference_latent_label_transfer_metrics.h5ad


In [13]:
process_datasets(['scPoli'], concatenated_dataset_path='concatenated_datasets/PDAC_concat_hvg_batch_key_datasets_hvg.h5ad',
extension='dataset_reference_latent_label_transfer.h5ad')

Processing scPoli_dataset_reference_latent_label_transfer.h5ad
AnnData object with n_obs × n_vars = 837983 × 4260
    obs: 'Dataset', 'ID', 'pseudo_barcode_two', 'Level 1 Annotation', 'Level 2 Annotation', 'Level 3 Annotation', 'Cell_type', 'scanvi_labels', 'Label_Harmonized', 'conditions_combined', 'cell_type_pred', 'cell_type_uncert', 'classifier_outcome', 'batch', 'Label_Harmonized_pred', 'Label_Harmonized_uncert', 'query', 'scpoli_labels', 'Condition', 'cell_barcode'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'neighbors', 'output_type', 'umap'
    obsm: 'X_cnv', 'X_cnv_pca', 'X_cnv_umap', 'X_harmony', 'X_pca', 'X_umap', 'X_emb', 'X_umap_emb', 'X_umap_raw'
    layers: 'counts', 'raw'
    obsp: 'connectivities', 'distances'
Saving at datasets_metrics/scPoli/scPoli_dataset_reference_latent_label_transfer_metrics.h5ad


In [14]:
process_datasets(['scPoli'], concatenated_dataset_path='concatenated_datasets/PDAC_concat_hvg_batch_key_datasets_hvg_8000.h5ad',
extension='_8000_label_transfer.h5ad')

Processing scPoli_dataset_reference_latent_8000_label_transfer.h5ad
AnnData object with n_obs × n_vars = 837983 × 8000
    obs: 'Dataset', 'ID', 'pseudo_barcode_two', 'Level 1 Annotation', 'Level 2 Annotation', 'Level 3 Annotation', 'Cell_type', 'scanvi_labels', 'Label_Harmonized', 'conditions_combined', 'cell_type_pred', 'cell_type_uncert', 'classifier_outcome', 'batch', 'Label_Harmonized_pred', 'Label_Harmonized_uncert', 'query', 'scpoli_labels', 'Condition', 'cell_barcode'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'neighbors', 'umap', 'output_type'
    obsm: 'X_cnv', 'X_cnv_pca', 'X_cnv_umap', 'X_harmony', 'X_pca', 'X_umap', 'X_emb', 'X_umap_emb', 'X_umap_raw'
    layers: 'counts', 'raw'
    obsp: 'connectivities', 'distances'
Saving at datasets_metrics/scPoli/scPoli_dataset_reference_latent_8000_label_transfer_metrics.h5ad


In [6]:
source = '/home/aih/shrey.parikh/PDAC/PDAC/datasets_metrics'
target = '/home/aih/shrey.parikh/PDAC/PDAC/scIB_metrics_toolbox'

In [10]:
copy_tree(source,target)

['/home/aih/shrey.parikh/PDAC/PDAC/scIB_metrics_toolbox/scPoli/scPoli_dataset_reference_latent_label_transfer_metrics.h5ad',
 '/home/aih/shrey.parikh/PDAC/PDAC/scIB_metrics_toolbox/scPoli/scPoli_dataset_reference_latent_8000_label_transfer_metrics.h5ad',
 '/home/aih/shrey.parikh/PDAC/PDAC/scIB_metrics_toolbox/scPoli/scPoli_donor_reference_latent_label_transfer_metrics.h5ad',
 '/home/aih/shrey.parikh/PDAC/PDAC/scIB_metrics_toolbox/scPoli/scPoli_donor_reference_latent_metrics.h5ad',
 '/home/aih/shrey.parikh/PDAC/PDAC/scIB_metrics_toolbox/scPoli/scPoli_dataset_reference_latent_8000_metrics.h5ad',
 '/home/aih/shrey.parikh/PDAC/PDAC/scIB_metrics_toolbox/scPoli/scPoli_dataset_reference_latent_metrics.h5ad',
 '/home/aih/shrey.parikh/PDAC/PDAC/scIB_metrics_toolbox/sysVI/sysVI_dataset_reference_latent_8000_metrics.h5ad',
 '/home/aih/shrey.parikh/PDAC/PDAC/scIB_metrics_toolbox/sysVI/sysVI_donor_reference_latent_metrics.h5ad',
 '/home/aih/shrey.parikh/PDAC/PDAC/scIB_metrics_toolbox/sysVI/sysVI_da

In [2]:
def process_datasets_scanvi(directories, concatenated_dataset_path, extension):
    adata_dataset = sc.read_h5ad(concatenated_dataset_path)
    for directory in directories:
        for file in os.listdir(directory):
            adata_training = adata_dataset.copy()
            if file.endswith(extension):
                file_path = os.path.join(directory, file)
                target_dir = os.path.join('datasets_metrics/scANVI', directory)
                if not os.path.exists(target_dir):
                    os.makedirs(target_dir)
                save_path = os.path.join(target_dir, file.replace('.h5ad', '_metrics.h5ad'))
                print(f'Processing {file_path.split('/')[1]}')
                adata = sc.read_h5ad(file_path)
                ID = adata_training.obs.ID.tolist()
                barcode = adata_training.obs_names.tolist()
                adata_training.obs_names = adata_training.obs.pseudo_barcode
                adata.obs_names = adata.obs.pseudo_barcode
                adata.obs_names = adata.obs_names.astype(str)
                adata.obs_names_make_unique()
                adata = adata[adata_training.obs_names, :]
                # adata = adata[~adata.obs.Dataset.str.contains('prototype')]
                adata_training.obsm['X_emb'] = adata.X
                adata_training.obsm['X_umap_emb'] = adata.obsm['X_umap']
                adata_training.obsm['X_umap_raw'] = adata_training.obsm['X_umap']
                # adata.obs['scpoli_labels'] = adata.obs.Label_Harmonized.fillna('Unknown')
                # adata.obs = adata.obs.astype(str)
                # adata.obs.loc[adata.obs['scpoli_labels'] == 'Unknown', 'scpoli_labels'] = adata.obs.loc[adata.obs['scpoli_labels'] == 'Unknown', 'cell_type_pred']
                adata_training.obs = adata.obs.copy()
                adata_training.uns = adata.uns.copy()
                adata_training.obsp = adata.obsp.copy()
                adata_training.uns['output_type'] = 'embed'
                print('adding condition')
                adata_training.obs['Condition'] = np.where(adata_training.obs.Dataset == 'Regev', 'snRNA-seq', 'scRNA-seq')
                adata_training.obs_names_make_unique()
                adata_training.var_names_make_unique()
                adata_training.obs['ID'] = ID
                adata_training.obs['cell_barcode'] = barcode
                # adata_training.obs = adata_training.obs.astype(str)
                adata_training.obs.set_index('pseudo_barcode', inplace=True)
                # adata_training.obs.rename(columns={'pseudo_barcode': 'pseudo_barcode_two'}, inplace=True)
                keys_to_keep = ['X_emb', 'X_umap_emb', 'X_umap_raw']
                keys_to_delete = [key for key in adata.obsm.keys() if key not in keys_to_keep]
                dict_map = dict(list(zip(adata_training.obs_names, adata_training.obs.scanvi_labels)))
                pickle_path = os.path.join(target_dir, 'dict_map.pickle')
                with open(pickle_path, 'wb') as handle:
                    pickle.dump(dict_map, handle, protocol=pickle.HIGHEST_PROTOCOL)
                for key in keys_to_delete:
                    del adata.obsm[key]
                
                print(f'Saving at {save_path}')
                print(adata_training)
                adata_training.write(save_path)

In [3]:
process_datasets_scanvi(['scanvae'], 'concatenated_datasets/PDAC_concat_hvg_batch_key_datasets_hvg.h5ad', extension='.h5ad')

Processing scanvae_dataset_full_latent.h5ad
adding condition
Saving at datasets_metrics/scANVI/scanvae/scanvae_dataset_full_latent_metrics.h5ad
AnnData object with n_obs × n_vars = 837983 × 4260
    obs: 'Label_Harmonized', 'Dataset', 'predictions', 'leiden', 'scanvi_labels', 'Condition', 'ID', 'cell_barcode'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'leiden', 'neighbors', 'umap', 'output_type'
    obsm: 'X_cnv', 'X_cnv_pca', 'X_cnv_umap', 'X_harmony', 'X_pca', 'X_umap', 'X_emb', 'X_umap_emb', 'X_umap_raw'
    layers: 'counts', 'raw'
    obsp: 'connectivities', 'distances'
Processing scanvae_dataset_full_latent_sep.h5ad
adding condition
Saving at datasets_metrics/scANVI/scanvae/scanvae_dataset_full_latent_sep_metrics.h5ad
AnnData object with n_obs × n_vars = 837983 × 4260
    obs: 'predictions_source', 'predictions', 'Label_Harmonized', 'batch', 'Dataset', 'leiden', 'scanvi_labels', 'Condi

## check the adata

In [4]:
adata = sc.read_h5ad('datasets_metrics/scANVI/scanvae_dataset_full_latent_metrics.h5ad')
adata.obs

,Label_Harmonized,Dataset,predictions,leiden,scanvi_labels,Condition,ID,cell_barcode
pseudo_barcode,,,,,,,,
non_annotated_cell_1,Unknown,Ding,Myeloid Cell,1,Myeloid Cell,scRNA-seq,HT056P1,AAACCCAAGTCATCGT-1-HT056P-S1PAA1-0-0-0
non_annotated_cell_2,Unknown,Ding,CAF,7,CAF,scRNA-seq,HT056P1,AAACCCACAACCACGC-1-HT056P-S1PAA1-0-0-0
non_annotated_cell_3,Unknown,Ding,T Cell,3,T Cell,scRNA-seq,HT056P1,AAACCCACAACTCCCT-1-HT056P-S1PAA1-0-0-0
non_annotated_cell_4,Unknown,Ding,T Cell,3,T Cell,scRNA-seq,HT056P1,AAACCCACAGACAATA-1-HT056P-S1PAA1-0-0-0
non_annotated_cell_5,Unknown,Ding,T Cell,3,T Cell,scRNA-seq,HT056P1,AAACCCACATGCCGAC-1-HT056P-S1PAA1-0-0-0
...,...,...,...,...,...,...,...,...
annotated_cell_253801,Macrophage,Zenodo_OUGS,Myeloid Cell,34,Macrophage,scRNA-seq,pk210318,pk210318_TTTGGTTTCCGATCTC-1
annotated_cell_253802,Macrophage,Zenodo_OUGS,Myeloid Cell,34,Macrophage,scRNA-seq,pk210318,pk210318_TTTGGTTTCCTTCACG-1
annotated_cell_253803,T Cell,Zenodo_OUGS,T Cell,3,T Cell,scRNA-seq,pk210318,pk210318_TTTGTTGCAATGCAAA-1


In [5]:
adata.obs.groupby('Condition').size()

Condition
scRNA-seq    645939
snRNA-seq    192044
dtype: int64

In [4]:
adata.obs['Condition'] = np.where(adata.obs.Dataset == 'Regev', 'snRNA-seq', 'scRNA-seq')


NameError: name 'adata' is not defined

In [ ]:
adata

# Add scpoli labels

In [ ]:
import pickle
with open('/home/aih/shrey.parikh/PDAC/PDAC/scPoli/label_barcode_dict.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

# Now, loaded_dict contains the dictionary that was saved
print(loaded_dict)

In [10]:
pwd

'/home/aih/shrey.parikh/PDAC/PDAC/datasets_metrics'

In [ ]:
# dicts created with pseudo barcodes and labels

In [8]:
dict_paths = {
    'scPoli_dataset_reference_latent_label_transfer_dict.pkl': '/home/aih/shrey.parikh/PDAC/PDAC/scPoli/scPoli_dataset_reference_latent_label_transfer_dict.pkl',
    'scPoli_donor_reference_latent_label_transfer_dict.pkl': '/home/aih/shrey.parikh/PDAC/PDAC/scPoli/scPoli_donor_reference_latent_label_transfer_dict.pkl',
    'scPoli_dataset_reference_latent_label_transfer_dict_8000.pkl': '/home/aih/shrey.parikh/PDAC/PDAC/scPoli/scPoli_dataset_reference_latent_label_transfer_dict_8000.pkl'
}

loaded_dicts = {}
for dict_name, dict_path in dict_paths.items():
    with open(dict_path, 'rb') as f:
        loaded_dicts[dict_name] = pickle.load(f)
print(loaded_dict)

# Function to determine which dictionary to use
def get_appropriate_dict(file_name):
    if 'donor_reference_latent_metrics.h5ad' in file_name:

        return loaded_dicts['scPoli_donor_reference_latent_label_transfer_dict.pkl']
    elif 'dataset_reference_latent_metrics_8000.h5ad' in file_name:
        return loaded_dicts['scPoli_dataset_reference_latent_label_transfer_dict_8000.pkl']
    elif 'dataset_reference_latent_metrics.h5ad' in file_name:
        return loaded_dicts['scPoli_dataset_reference_latent_label_transfer_dict.pkl']
    return None

# Process the files
count = 0
for directory in ['scVI', 'sysVI', 'scPoli']:
    for file in os.listdir(directory):
        if file.endswith(('donor_reference_latent_metrics.h5ad', 'dataset_reference_latent_metrics_8000.h5ad', 'dataset_reference_latent_metrics.h5ad')) and not file.startswith('test'):
            count += 1
            file_path = os.path.join(directory, file)
            print(f'Processing {file_path}')
            adata = sc.read_h5ad(file_path)
            appropriate_dict = get_appropriate_dict(file)
            if appropriate_dict:
                adata.obs['scpoli_labels'] = adata.obs.pseudo_barcode.map(appropriate_dict).fillna('Unknown')
                adata.write(file_path)
                print(f'Updated scpoli_labels for {file_path}')

print(f'Total files processed: {count}')

Processing scVI_donor_reference_latent_metrics.h5ad
AnnData object with n_obs × n_vars = 837983 × 3781
    obs: 'ID', 'Dataset', 'Condition', 'leiden', 'scanvi_labels', 'pseudo_barcode', 'scpoli_labels'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'leiden', 'neighbors', 'output_type', 'umap'
    obsm: 'X_emb', 'X_umap_emb', 'X_umap_raw'
    layers: 'counts', 'raw'
    obsp: 'connectivities', 'distances'
1


### same done for scanvi labels

### repeated for scVI and sysVI


In [ ]:
process_datasets(['scVI', 'sysVI'], concatenated_dataset_path='concatenated_datasets/PDAC_concat_hvg_batch_key_datasets_hvg.h5ad',
extension='*.h5ad')

In [8]:
with open('datasets_metrics/scANVI/scanvae/dict_map.pickle', 'rb') as handle:
    dict_map = pickle.load(handle)

In [17]:
def label_transfer(adata_path, dict_map):
        adata = sc.read_h5ad(adata_path)
        adata.obs['scanvi_labels'] = adata.obs.pseudo_barcode.map(dict_map)
        print(f'Saving {adata_path}')
        adata.write(adata_path)

In [19]:
for file in os.listdir('datasets_metrics/scVI'):
    if 'dataset' in file:
        file_path = os.path.join('datasets_metrics/scVI', file)
        print(file_path)
        label_transfer(file_path, dict_map)

datasets_metrics/scVI/scVI_dataset_reference_latent_8000_metrics.h5ad
Saving datasets_metrics/scVI/scVI_dataset_reference_latent_8000_metrics.h5ad
datasets_metrics/scVI/scVI_dataset_reference_latent_metrics.h5ad
Saving datasets_metrics/scVI/scVI_dataset_reference_latent_metrics.h5ad


In [21]:
for file in os.listdir('datasets_metrics/sysVI'):
    if 'dataset' in file:
        file_path = os.path.join('datasets_metrics/sysVI', file)
        print(file_path)
        label_transfer(file_path, dict_map)

datasets_metrics/sysVI/sysVI_dataset_reference_latent_8000_metrics.h5ad
Saving datasets_metrics/sysVI/sysVI_dataset_reference_latent_8000_metrics.h5ad
datasets_metrics/sysVI/sysVI_dataset_reference_latent_metrics.h5ad
Saving datasets_metrics/sysVI/sysVI_dataset_reference_latent_metrics.h5ad


In [22]:
test = sc.read_h5ad('datasets_metrics/sysVI/sysVI_dataset_reference_latent_8000_metrics.h5ad')
test.obs

,n_genes,ID,n_counts,log_counts,mt_frac,Dataset,sampleid,scrublet_scores,pid,n_genes_by_counts,...,exocrine_score_1,activated_stroma_score_1,histone_score_1,normal_stroma_score_1,olfactory_score_1,na_score_1,Condition,scanvi_labels,pseudo_barcode,scpoli_labels
AAACCCAAGTCATCGT-1-HT056P-S1PAA1-0-0-0,2929.0,HT056P1,12375.0,9.423433,0.096727,Ding,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,scRNA,Myeloid Cell,non_annotated_cell_1,Macrophage
AAACCCACAACCACGC-1-HT056P-S1PAA1-0-0-0,3440.0,HT056P1,10981.0,9.303922,0.068573,Ding,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,scRNA,CAF,non_annotated_cell_2,Fibroblast
AAACCCACAACTCCCT-1-HT056P-S1PAA1-0-0-0,1976.0,HT056P1,5352.0,8.585226,0.113416,Ding,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,scRNA,T Cell,non_annotated_cell_3,T Cell
AAACCCACAGACAATA-1-HT056P-S1PAA1-0-0-0,2446.0,HT056P1,7301.0,8.895766,0.151760,Ding,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,scRNA,T Cell,non_annotated_cell_4,Natural killer
AAACCCACATGCCGAC-1-HT056P-S1PAA1-0-0-0,2675.0,HT056P1,8489.0,9.046527,0.112852,Ding,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,scRNA,T Cell,non_annotated_cell_5,T Cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pk210318_TTTGGTTTCCGATCTC-1,NaN,pk210318,NaN,NaN,NaN,Zenodo_OUGS,NaN,NaN,NaN,NaN,...,-0.072897,-0.017155,0.079092,-0.005438,-0.001765,-0.013295,scRNA,Macrophage,annotated_cell_253801,Macrophage
pk210318_TTTGGTTTCCTTCACG-1,NaN,pk210318,NaN,NaN,NaN,Zenodo_OUGS,NaN,NaN,NaN,NaN,...,-0.036612,-0.036467,0.058162,0.011655,0.006083,-0.010507,scRNA,Macrophage,annotated_cell_253802,Macrophage
pk210318_TTTGTTGCAATGCAAA-1,NaN,pk210318,NaN,NaN,NaN,Zenodo_OUGS,NaN,NaN,NaN,NaN,...,-0.089159,-0.111194,-0.019965,-0.030641,-0.048538,-0.018958,scRNA,T Cell,annotated_cell_253803,T Cell
pk210318_TTTGTTGGTATGAGCG-1,NaN,pk210318,NaN,NaN,NaN,Zenodo_OUGS,NaN,NaN,NaN,NaN,...,-0.048846,-0.054666,-0.018734,-0.024168,-0.042796,0.002950,scRNA,T Cell,annotated_cell_253804,T Cell
